In [47]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import math

In [48]:
df = pd.read_csv("./dataset/20170827_mawilab_flow_000.csv",sep=',')
df.head() 

,sIP,dIP,sPort,dPort,pro,packets,bytes,flags,sTime,durat,...,appli,Records,%Records,cumul_%,status,taxonomy,label,heuristic,distance,nbDetectors
0,70.37.209.160,203.205.84.19,0,0,1,1,32,NaN,2017/08/27T05:00:00.290,0.0,...,0,normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,118.150.224.119,163.158.254.254,80,14521,6,1,48,S A,2017/08/27T05:00:00.290,0.0,...,0,normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,203.104.241.234,149.20.207.139,22,51316,6,1,1440,A,2017/08/27T05:00:00.290,0.0,...,0,normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,164.122.96.252,203.205.73.175,56670,80,6,1,52,A,2017/08/27T05:00:00.290,0.0,...,0,anomaly,mptpHTTP,Multi Points,0.39596,8.0,anomalous,NaN,NaN,NaN
4,164.122.96.252,203.205.73.175,56670,80,6,1,52,A,2017/08/27T05:00:00.290,0.0,...,0,anomaly,mptpHTTP,Multi Points,0.39596,8.0,anomalous,NaN,NaN,NaN


In [49]:
df.loc[:, 'U'] = pd.Series(np.zeros(df.shape[0]),dtype='int8')
df.loc[:, 'N'] = pd.Series(np.zeros(df.shape[0]),dtype='int8')
new_flag_df = df['flags'].to_frame()
new_flag_df = new_flag_df['flags'].str.get_dummies(' ')
new_flag_df.head()

,A,C,E,EC,F,PA,R,S
0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0


In [50]:
"""
1. Pro
2. Packets
3. Bytes
4. Flags
5. Durat
6. Records
7. svc_port - combination of sPort and dPort

v = np.minimum(df['sPort'], df['dPort'])
df['svc_port'] = np.where(v <= 1024, v, df['dPort'])
"""
df['svc_port'] = df.apply(lambda x : min(x['sPort'],x['dPort']) if min(x['sPort'],x['dPort']) <= 1024 else x['dPort'], axis=1)
df = df.loc[:,('pro', 'packets', 'bytes', 'svc_port', 'durat','U','N', 'Records')]
df.fillna(0, inplace=True) 
new_df = df.drop(['Records'], axis=1)
mawii_labels = pd.get_dummies(df['Records'],prefix='',prefix_sep='')
mawii_features = pd.concat([new_df, new_flag_df], axis=1)
print(mawii_labels.head())
print(mawii_features.head())

   anomaly  normal
0        0       1
1        0       1
2        0       1
3        1       0
4        1       0
   pro  packets  bytes  svc_port  durat  U  N  A  C  E  EC  F  PA  R  S
0    1        1     32         0    0.0  0  0  0  0  0   0  0   0  0  0
1    6        1     48        80    0.0  0  0  1  0  0   0  0   0  0  1
2    6        1   1440        22    0.0  0  0  1  0  0   0  0   0  0  0
3    6        1     52        80    0.0  0  0  1  0  0   0  0   0  0  0
4    6        1     52        80    0.0  0  0  1  0  0   0  0   0  0  0


In [51]:
mawii_features = mawii_features.as_matrix()
mawii_labels = mawii_labels.as_matrix()

In [52]:
"""
# Converting to Tensor: 
mawii_features = tf.convert_to_tensor(mawii_features, dtype=)
mawii_labels = tf.convert_to_tensor(mawii_labels)
"""

'\n# Converting to Tensor: \nmawii_features = tf.convert_to_tensor(mawii_features, dtype=)\nmawii_labels = tf.convert_to_tensor(mawii_labels)\n'

In [53]:
def conv_2d(x,W): 
    # Padding : surround the original image with 0's 
    # valid padding - no padding.size of the image shrinks  resultant image : (n - f + 1, n - f + 1)
    # same padding - resultant is same as that of original. resultant image : (n + 2p - f + 1, n +2p - f + 1)  p = (f - 1)/2
    # Strides : ((n + 2p - f)/s + 1, (n +2p - f)/s + 1)
    return tf.nn.conv2d(input = x, filter = W, strides = [1,1,1,1], padding="SAME")

def avg_pool_2x2(x):
    # MAX/AVG Pooling : ((n + 2p - f)/s + 1, (n +2p - f)/s + 1)
    return tf.nn.avg_pool(x, k_size=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

In [54]:
# cnn-1,2,3 model 

In [55]:
def conv1_network(x_image, reuse=False): 
    with tf.variable_scope('ConvNet_1') as scope: 
        if reuse:
             tf.get_variable_scope().reuse_variables()
        # First Conv and Pooling layers
        W_conv1 = tf.get_variable('c_wconv1',[5,5,1,8],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv1 = tf.get_variable('c_bconv1',[8],initializer=tf.constant_initializer(0))
        h_conv1 = tf.nn.relu(conv_2d(x_image, W_conv1) + b_conv1)
        h_pool1 = avg_pool_2x2(h_conv1)
        
        # Fully Connected layer1 
        # First fully connected layers
        W_fc1 = tf.get_variable('d_wfc1', [7*7*8,16],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc1 = tf.get_variable('d_bfc1', [16], initializer=tf.constant_initializer(0))
        h_pool_flat = tf.reshape(h_pool1, [-1,7*7*8])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat,W_fc1)+b_fc1)
        
        #Seconf fully connected layers
        W_fc2 = tf.get_variable('d_wfc2', [16,1],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc2 = tf.get_variable('d_bfc2', [1], initializer=tf.constant_initializer(0))

        y_conv = tf.nn.relu(tf.matmul(h_fc1,W_fc2)+b_fc2)
    
    return y_conv 

In [56]:
def batches(batch_size, features, labels): 
    assert len(features) == len(labels)
    sample_size = len(features)
    output_batches = [] 
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        output_batches.append(batch)
    return output_batches
def print_epochs_cost_accuracy(epoch, sess, batch_features, batch_labels): 
    costVal = sess.run(cost, feed_dict = {features: batch_features, labels:batch_labels})
    AccuracyVal = sess.run(accuracy,feed_dict = {features: valid_features, labels : valid_labels})
    print("epoch {}, cost {}, Accuracy {} ".format(epoch, costVal, AccuracyVal))

In [57]:
n_input = 15
n_classes = 2

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input, 1])
labels = tf.placeholder(tf.float32, [None, n_classes])



keep_prob = 0.5
learning_rate = 0.01
output = tf.nn.dropout(conv1_network(features,reuse=True), keep_prob)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = output, labels = labels))

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(output,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

batchSize =  128 

epochs = 1000


init = tf.global_variables_initializer() 

with tf.Session() as sess:
    sess.run(init) 
    total_batches = batches(batchSize, mawii_features, mawii_labels)
    
    for epoch in range(epochs): 
        for batch_features, batch_labels in total_batches: 
            train_data = {features: batch_features, labels : batch_labels, keep_prob : 0.7}
            sess.run(optimizer, feed_dict = train_data)
        # Print status for every 10 epochs
        if epoch % 100 == 0:
            valid_accuracy = sess.run(
                accuracy,
                feed_dict={
                    features: mawii_features,
                    labels: mawii_labels,
                    keep_prob : 0.7})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(
                epoch,
                valid_accuracy))
    Accuracy = sess.run(accuracy, feed_dict={features : mawii_features, labels :mawii_labels, keep_prob : 0.7})
    print('Trained Model Saved.')
print("Accuracy value is {}".format(Accuracy))

ValueError: Shape must be rank 4 but is rank 2 for 'ConvNet_1_1/Conv2D' (op: 'Conv2D') with input shapes: [?,15], [5,5,1,8].

In [ ]:
def conv2_network(x_image, reuse=False): 
    with tf.variable_scope('ConvNet2') as scope: 
        if reuse:
             tf.get_variable_scope().reuse_variables()
        # First Conv and Pooling layers
        W_conv1 = tf.get_variable('c_wconv1',[5,5,1,8],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv1 = tf.get_variable('c_bconv1',[8],initializer=tf.constant_initializer(0))
        h_conv1 = tf.nn.relu(conv_2d(x_image, W_conv1) + b_conv1)
        h_pool1 = avg_pool_2x2(h_conv1)
        
        # Seond Conv and Pooling layers
        W_conv2 = tf.get_variable('d_wconv2',[5,5,8,16],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv2 = tf.get_variable('d_bconv2',[16],initializer=tf.constant_initializer(0))
        h_conv2 = tf.nn.relu(conv_2d(h_pool1, W_conv2) + b_conv2)
        h_pool2 = avg_pool_2x2(h_conv2)
        
        # First fully connected layers
        W_fc1 = tf.get_variable('d_wfc1', [7*7*16,32],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc1 = tf.get_variable('d_bfc1', [32], initializer=tf.constant_initializer(0))
        h_pool_flat = tf.reshape(h_pool2, [-1,7*7*16])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat,W_fc1)+b_fc1)
        
        #Seconf fully connected layers
        W_fc2 = tf.get_variable('d_wfc2', [32,1],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc2 = tf.get_variable('d_bfc2', [1], initializer=tf.constant_initializer(0))

        y_conv = tf.nn.relu(tf.matmul(h_fc1,W_fc2)+b_fc2)
    
    return y_conv
        

In [ ]:
def conv3_network(x_image, reuse=False): 
    with tf.variable_scope('ConvNet3') as scope: 
        if reuse:
             tf.get_variable_scope().reuse_variables()
        # First Conv and Pooling layers
        W_conv1 = tf.get_variable('c_wconv1',[5,5,1,8],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv1 = tf.get_variable('c_bconv1',[8],initializer=tf.constant_initializer(0))
        h_conv1 = tf.nn.relu(conv_2d(x_image, W_conv1) + b_conv1)
        h_pool1 = avg_pool_2x2(h_conv1)
        
        # Seond Conv and Pooling layers
        W_conv2 = tf.get_variable('d_wconv2',[5,5,8,16],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv2 = tf.get_variable('d_bconv2',[16],initializer=tf.constant_initializer(0))
        h_conv2 = tf.nn.relu(conv_2d(h_pool1, W_conv2) + b_conv2)
        h_pool2 = avg_pool_2x2(h_conv2)
        
        # Third Conv and Pooling layers
        W_conv3 = tf.get_variable('d_wconv2',[5,5,16,32],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv3 = tf.get_variable('d_bconv2',[32],initializer=tf.constant_initializer(0))
        h_conv3 = tf.nn.relu(conv_2d(h_pool2, W_conv3) + b_conv3)
        h_pool3 = avg_pool_2x2(h_conv3)
        
        # First fully connected layers
        W_fc1 = tf.get_variable('d_wfc1', [7*7*32,64],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc1 = tf.get_variable('d_bfc1', [64], initializer=tf.constant_initializer(0))
        h_pool_flat = tf.reshape(h_pool3, [-1,7*7*32])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat,W_fc1)+b_fc1)
        
        #Seconf fully connected layers
        W_fc2 = tf.get_variable('d_wfc2', [64,1],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc2 = tf.get_variable('d_bfc2', [1], initializer=tf.constant_initializer(0))

        y_conv = tf.nn.relu(tf.matmul(h_fc1,W_fc2)+b_fc2)
    
    return y_conv